In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

In [2]:
hidden_size = 100

class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content))
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        self.ch_to_idx = {ch:i for i,ch in enumerate(self.vocab)}
        self.idx_to_ch = {i:ch for i,ch in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.lines[index]
        x_str = ' ' + line # add a space at the beginning, which indicates a vector of zeros.
        y_str = line + '\n'
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        
        y[0] = self.ch_to_idx[y_str[0]]
        #we start from the second character because the first character of x was nothing(vector of zeros).
        for i, (x_ch, y_ch) in enumerate(zip(x_str[1:], y_str[1:]), 1):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [3]:
class RNN(nn.Module):
    def __init__(self, hidden_size, input_size, output_size):
        super().__init__()
        # TO DO
        # YOUR CODE HERE
        self.linearax = nn.Linear(input_size, hidden_size) #a2x
        self.linearaa = nn.Linear(hidden_size, hidden_size) #a2a
        self.linearay = nn.Linear(hidden_size, output_size) #h2o

    def forward(self, x, a):
        # TO DO
        # YOUR CODE HERE
        z1 = self.linearax(x)
        z2 = self.linearaa(a)
        z = z1 + z2
        a_prime = torch.tanh(z)
        y_pred = self.linearay(a_prime)
        
        return a_prime, y_pred

In [4]:
def sample(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['\n']
    indices = []
    pred_char_idx = -1
    
    list_for_idx = []
    for i in range (27):
        list_for_idx.append(i)
        
    # Step 1: initialize first input and hidden state
    # YOUR CODE HERE
    a_prev = torch.zeros(1, 100)
    x = torch.zeros(27)

    
    with torch.no_grad():
        while pred_char_idx != newline_idx and word_size != 50:
            # Step 2: Forward propagate x using the equations (1), (2) and (3)
            # YOUR CODE HERE
            a, y_pred = model.forward(x, a_prev)
            y_softmax = F.softmax(y_pred)
    
            # Step 3: Sample the index of a character within the vocabulary from the probability distribution
            # YOUR CODE HERE
            idx = np.random.choice(list_for_idx, p = y_softmax.reshape(27).numpy())
            indices.append(idx)
            
            # Step 4: Overwrite the input character as the one corresponding to the sampled index.
            # YOUR CODE HERE
            x = torch.zeros(27)
            x[idx] = 1
            
            pred_char_idx = idx 
            word_size += 1
            
            a_prev = a
            
        if word_size == 50:
            indices.append(newline_idx)
            
    return indices

def print_sample(sample_idxs):
    print(trn_ds.idx_to_ch[sample_idxs[0]].upper(), end='')
    [print(trn_ds.idx_to_ch[x], end='') for x in sample_idxs[1:]]

In [5]:
def train_one_epoch(model, loss_fn, optimizer):
    # Go through the training examples one at a time
    for line_num, (x, y) in enumerate(trn_dl):
        model.train()
        loss = 0
        optimizer.zero_grad()
        
        # Initialize parameters
        # YOUR CODE HERE
        a = torch.zeros(100)
        
        for i in range(x.shape[1]):
            # Forward propagate through the RNN to compute the loss
            # YOUR CODE HERE
            a, y_pred = model.forward(x[0][i], a)
            loss += loss_fn(y_pred.view(1, -1), y[:,i])            
            
        #Every 100 steps of stochastic gradient descent, print one sampled name to see how the algorithm is doing
        if (line_num+1) % 100 == 0:
            # YOUR CODE HERE
            # HINT: print_sample()
            print_sample(sample(model))
                
        # Backpropagate through time
        # YOUR CODE HERE
        
        loss.backward()
        
        # Clip your gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        
        # Update parameters
        # YOUR CODE HERE
        optimizer.step()
        

In [6]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, batch_size=1, shuffle=True)

def train(trn_ds, trn_dl, epochs=1):
    # Create a new model, loss_fn and optimizer.
    # YOUR CODE HERE
    model = RNN(100, 27, 27)
    # Use cross entropy loss
    loss_fn = nn.CrossEntropyLoss()
    # Use Adam
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    for e in range(1, epochs+1):
        print(f'{"-"*20} Epoch {e} {"-"*20}')
        train_one_epoch(model, loss_fn, optimizer)

In [8]:
#Start training
train(trn_ds, trn_dl, epochs=50)

-------------------- Epoch 1 --------------------


c:\users\brian\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Ohtocouhaur
Ceefntnrisasrur
Ntecnnolonrnyros
Knlticatdusgurii
Ayeiekalodanuoixos
Dehainagiar
Npaanmssuuus
Grotahous
Hmawzosautus
Byamirawopa
Utausulas
Murosausus
Taprlosourus
Maltancnnue

-------------------- Epoch 2 --------------------
Hyjomonan
Starahorosalros
Tamanntadabin
Psolosausus
Itasaurus
Heloicalatops
Lapanoleus
Cvpiasuurus
Nmynesaurus
Pyyingochescur
Tolaciibentop
Pptiaosaurus
Siugonliaur
Bdekasaurus
Ruhinobaurts
-------------------- Epoch 3 --------------------
Chueracyastolosaurus
Alocrnthos
Eloradrosaurus
Cauroploma
Anwusanius
Aurosaurus
Harakeosaurus
Audonisaurus
Angieosaurus
Dantradran
Unujiiaucesaurus
Piapiston
Chibenosnusus
Yberopanthonsaur
Odinocenotopharophites
-------------------- Epoch 4 --------------------
Leieenosaurus
Trillongisaus
Gunelicerur
Jphoherytoter
Bataorroshurus
Eucoprosaurus
Mermodosalrus
Wucristilus
Sultavinaus
Ymusalansaurus
Kirenodonns
Oseusanicsuunus
Harrosaurus
Miteechrug
Pholoroptodyl
-------------------- Epoch 5 --------------------
Bolusauru